# Stage_1
calculating the win/loss ratio's correlation with the population of the city it is in for the NHL using 2018 data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [2]:
nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [3]:
nhl_df=nhl_df.drop([0,9,18,26])
nhl_df.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL
5,Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL


In [4]:
cities['NHL']=cities['NHL'].str.replace('\[.*\]','')
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [5]:
nhl_df=nhl_df[nhl_df['year'] == 2018]
nhl_df.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL
5,Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL


In [6]:
nhl_df[['Metropolitan area','nhl team']]=nhl_df['team'].str.rsplit(n=1,expand=True)


In [7]:
nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].replace({'Columbus Blue':'Columbus','Toronto Maple':'Toronto','Detroit Red':'Detroit','Vegas Golden':'Vegas','San Jose':'San Francisco Bay Area','Dallas':'Dallas–Fort Worth','Washington':'Washington, D.C.','Minnesota':'Minneapolis–Saint Paul','Florida':'Miami–Fort Lauderdale','Arizona':'Phoenix','Vegas':'Las Vegas','Carolina':'Raleigh','New Jersey':'New York City','Anaheim':'Los Angeles','Colorado':'Denver','New York':'New York City','Tampa Bay':'Tampa Bay Area'},regex=True)
nhl_df.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,Metropolitan area,nhl team
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL,Tampa Bay Area,Lightning*
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL,Boston,Bruins*
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL,Toronto,Leafs*
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL,Miami–Fort Lauderdale,Panthers
5,Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL,Detroit,Wings


In [8]:
nhl_df['W']=nhl_df['W'].astype(float)
nhl_df['L']=nhl_df['L'].astype(float)
print(nhl_df.dtypes)

team                  object
GP                    object
W                    float64
L                    float64
OL                    object
PTS                   object
PTS%                  object
GF                    object
GA                    object
SRS                   object
SOS                   object
RPt%                  object
ROW                   object
year                   int64
League                object
Metropolitan area     object
nhl team              object
dtype: object


In [9]:
nhl_df['W/L']=nhl_df['W']/(nhl_df['W']+nhl_df['L'])
nhl_df.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,Metropolitan area,nhl team,W/L
1,Tampa Bay Lightning*,82,54.0,23.0,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL,Tampa Bay Area,Lightning*,0.701299
2,Boston Bruins*,82,50.0,20.0,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL,Boston,Bruins*,0.714286
3,Toronto Maple Leafs*,82,49.0,26.0,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL,Toronto,Leafs*,0.653333
4,Florida Panthers,82,44.0,30.0,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL,Miami–Fort Lauderdale,Panthers,0.594595
5,Detroit Red Wings,82,30.0,39.0,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL,Detroit,Wings,0.434783


In [10]:
merge_df=pd.merge(cities,nhl_df,'outer',on='Metropolitan area')
merge_df=merge_df.drop(['NFL','MLB','NBA','GP','GF','GA','SRS','SOS','RPt%','ROW','year','League','nhl team','PTS','PTS%','OL','W','L','team'],axis=1)
merge_df.head()


,Metropolitan area,Population (2016 est.)[8],NHL,W/L
0,New York City,20153634,RangersIslandersDevils,0.602740
1,New York City,20153634,RangersIslandersDevils,0.486111
2,New York City,20153634,RangersIslandersDevils,0.465753
3,Los Angeles,13310447,KingsDucks,0.637681
4,Los Angeles,13310447,KingsDucks,0.608108


In [11]:
merge_df=merge_df.dropna()
merge_df=merge_df.reset_index(drop = True)
merge_df.head()

,Metropolitan area,Population (2016 est.)[8],NHL,W/L
0,New York City,20153634,RangersIslandersDevils,0.602740
1,New York City,20153634,RangersIslandersDevils,0.486111
2,New York City,20153634,RangersIslandersDevils,0.465753
3,Los Angeles,13310447,KingsDucks,0.637681
4,Los Angeles,13310447,KingsDucks,0.608108


In [12]:
merge_df=merge_df.groupby(['Metropolitan area','Population (2016 est.)[8]']).agg('mean')


# Stage_2

In [13]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [14]:
nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [15]:
nba_df['team']=nba_df['team'].str.replace('\*\s\(\d\)','')
nba_df.head()

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League
0,Toronto Raptors,59,23,0.72,—,111.7,103.9,7.29,2018,NBA
1,Boston Celtics,55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA
2,Philadelphia 76ers,52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA
3,Cleveland Cavaliers,50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA
4,Indiana Pacers,48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA


In [16]:
cities['NBA']=cities['NBA'].str.replace('\[.*\]','')
cities.head(30)

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls,Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals[note 10],Wizards,Capitals
6,Philadelphia,6070500,Eagles,Phillies[note 12],76ers,Flyers[note 13]
7,Boston,4794447,Patriots[note 14],Red Sox[note 15],Celtics,Bruins
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild[note 16]
9,Denver,2853077,Broncos,Rockies,Nuggets,Avalanche[note 18]


In [17]:
nba_df['team']=nba_df['team'].str.replace('\s\(.*\)','')
nba_df[['Metropolitan area','nba_team']]=nba_df['team'].str.rsplit(n=1,expand=True)
nba_df=nba_df[nba_df['year']==2018]
nba_df.head(20)

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League,Metropolitan area,nba_team
0,Toronto Raptors,59,23,0.72,—,111.7,103.9,7.29,2018,NBA,Toronto,Raptors
1,Boston Celtics,55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA,Boston,Celtics
2,Philadelphia 76ers,52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA,Philadelphia,76ers
3,Cleveland Cavaliers,50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA,Cleveland,Cavaliers
4,Indiana Pacers,48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA,Indiana,Pacers
5,Miami Heat,44,38,0.537,15.0,103.4,102.9,0.15,2018,NBA,Miami,Heat
6,Milwaukee Bucks,44,38,0.537,15.0,106.5,106.8,-0.45,2018,NBA,Milwaukee,Bucks
7,Washington Wizards,43,39,0.524,16.0,106.6,106.0,0.53,2018,NBA,Washington,Wizards
8,Detroit Pistons,39,43,0.47600000000000003,20.0,103.8,103.9,-0.26,2018,NBA,Detroit,Pistons
9,Charlotte Hornets,36,46,0.439,23.0,108.2,108.0,0.07,2018,NBA,Charlotte,Hornets


In [18]:
nba_df['Metropolitan area']=nba_df['Metropolitan area'].replace({'Brooklyn':'New York City','New York':'New York City','Golden State':'San Francisco Bay Area','Dallas':'Dallas–Fort Worth','Washington':'Washington, D.C.','Minnesota':'Minneapolis–Saint Paul','Miami':'Miami–Fort Lauderdale','Indiana':'Indianapolis','Portland Trail':'Portland','Utah':'Salt Lake City'},regex=True)
nba_df.head(20)

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League,Metropolitan area,nba_team
0,Toronto Raptors,59,23,0.72,—,111.7,103.9,7.29,2018,NBA,Toronto,Raptors
1,Boston Celtics,55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA,Boston,Celtics
2,Philadelphia 76ers,52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA,Philadelphia,76ers
3,Cleveland Cavaliers,50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA,Cleveland,Cavaliers
4,Indiana Pacers,48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA,Indianapolis,Pacers
5,Miami Heat,44,38,0.537,15.0,103.4,102.9,0.15,2018,NBA,Miami–Fort Lauderdale,Heat
6,Milwaukee Bucks,44,38,0.537,15.0,106.5,106.8,-0.45,2018,NBA,Milwaukee,Bucks
7,Washington Wizards,43,39,0.524,16.0,106.6,106.0,0.53,2018,NBA,"Washington, D.C.",Wizards
8,Detroit Pistons,39,43,0.47600000000000003,20.0,103.8,103.9,-0.26,2018,NBA,Detroit,Pistons
9,Charlotte Hornets,36,46,0.439,23.0,108.2,108.0,0.07,2018,NBA,Charlotte,Hornets


In [19]:
nba_df['W/L%']=nba_df['W/L%'].astype(float)
#nba_df['L']=nba_df['L'].astype(float)
print(nba_df.dtypes)

team                  object
W                     object
L                     object
W/L%                 float64
GB                    object
PS/G                  object
PA/G                  object
SRS                   object
year                   int64
League                object
Metropolitan area     object
nba_team              object
dtype: object


In [20]:
merge_df=pd.merge(cities,nba_df,'outer',on='Metropolitan area')
merge_df=merge_df.drop(['NFL','MLB','NHL','year','League','nba_team','PS/G','SRS','PA/G','W','L','team','GB'],axis=1)
merge_df.head(20)

,Metropolitan area,Population (2016 est.)[8],NBA,W/L%
0,New York City,20153634,KnicksNets,0.354
1,New York City,20153634,KnicksNets,0.341
2,Los Angeles,13310447,LakersClippers,0.512
3,Los Angeles,13310447,LakersClippers,0.427
4,San Francisco Bay Area,6657982,Warriors,0.707
5,Chicago,9512999,Bulls,0.329
6,Dallas–Fort Worth,7233323,Mavericks,0.293
7,"Washington, D.C.",6131977,Wizards,0.524
8,Philadelphia,6070500,76ers,0.634
9,Boston,4794447,Celtics,0.671


In [21]:
merge_df=merge_df.dropna()
merge_df=merge_df.reset_index(drop = True)
merge_df.head(20)

,Metropolitan area,Population (2016 est.)[8],NBA,W/L%
0,New York City,20153634,KnicksNets,0.354
1,New York City,20153634,KnicksNets,0.341
2,Los Angeles,13310447,LakersClippers,0.512
3,Los Angeles,13310447,LakersClippers,0.427
4,San Francisco Bay Area,6657982,Warriors,0.707
5,Chicago,9512999,Bulls,0.329
6,Dallas–Fort Worth,7233323,Mavericks,0.293
7,"Washington, D.C.",6131977,Wizards,0.524
8,Philadelphia,6070500,76ers,0.634
9,Boston,4794447,Celtics,0.671


In [22]:
merge_df=merge_df.groupby(['Metropolitan area','Population (2016 est.)[8]']).agg('mean')
merge_df.head(20)

,,W/L%
Metropolitan area,Population (2016 est.)[8],
Atlanta,5789700,0.2930
Boston,4794447,0.6710
Charlotte,2474314,0.4390
Chicago,9512999,0.3290
Cleveland,2055612,0.6100
Dallas–Fort Worth,7233323,0.2930
Denver,2853077,0.5610
Detroit,4297617,0.4760
Houston,6772470,0.7930


# Stage_3
#### calculating the win/loss ratio's correlation with the population of the city it is in for the MLB using 2018 data 

In [23]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]



In [24]:
mlb_df.head()

,team,W,L,W-L%,GB,year,League
0,Boston Red Sox,108,54,0.667,--,2018,MLB
1,New York Yankees,100,62,0.617,8.0,2018,MLB
2,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB


In [25]:
cities['MLB']=cities['MLB'].str.replace('\[.*\]','')
cities.head(55)

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets,KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals,Wizards[note 11],Capitals
6,Philadelphia,6070500,Eagles,Phillies,76ers,Flyers[note 13]
7,Boston,4794447,Patriots[note 14],Red Sox,Celtics,Bruins
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild[note 16]
9,Denver,2853077,Broncos,Rockies,Nuggets[note 17],Avalanche[note 18]


In [26]:
mlb_df[['Metropolitan area','mlb_team']]=mlb_df['team'].str.rsplit(n=1,expand=True)
mlb_df=mlb_df[mlb_df['year']==2018]
mlb_df.head()

,team,W,L,W-L%,GB,year,League,Metropolitan area,mlb_team
0,Boston Red Sox,108,54,0.667,--,2018,MLB,Boston Red,Sox
1,New York Yankees,100,62,0.617,8.0,2018,MLB,New York,Yankees
2,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB,Tampa Bay,Rays
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB,Toronto Blue,Jays
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB,Baltimore,Orioles


In [27]:
mlb_df['Metropolitan area']=mlb_df['Metropolitan area'].replace({'Boston Red':'Boston','Tampa Bay':'Tampa Bay Area','New York':'New York City','Texas':'Dallas–Fort Worth','Washington':'Washington, D.C.','San Francisco':'San Francisco Bay Area','Minnesota':'Minneapolis–Saint Paul','Miami':'Miami–Fort Lauderdale','Colorado':'Denver','Chicago White':'Chicago','Toronto Blue':'Toronto','Arizona':'Phoenix','Oakland':'San Francisco Bay Area'},regex=True)
mlb_df.head(40)

,team,W,L,W-L%,GB,year,League,Metropolitan area,mlb_team
0,Boston Red Sox,108,54,0.667,--,2018,MLB,Boston,Sox
1,New York Yankees,100,62,0.617,8.0,2018,MLB,New York City,Yankees
2,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB,Tampa Bay Area,Rays
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB,Toronto,Jays
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB,Baltimore,Orioles
5,Cleveland Indians,91,71,0.562,--,2018,MLB,Cleveland,Indians
6,Minnesota Twins,78,84,0.481,13.0,2018,MLB,Minneapolis–Saint Paul,Twins
7,Detroit Tigers,64,98,0.395,27.0,2018,MLB,Detroit,Tigers
8,Chicago White Sox,62,100,0.383,29.0,2018,MLB,Chicago,Sox
9,Kansas City Royals,58,104,0.358,33.0,2018,MLB,Kansas City,Royals


In [28]:
mlb_df['W-L%']=mlb_df['W-L%'].astype(float)

In [29]:
merge_df=pd.merge(cities,mlb_df,'outer',on='Metropolitan area')
merge_df=merge_df.drop(['NFL','NBA','NHL','year','League','mlb_team','W','L','team','GB'],axis=1)
merge_df.head(20)

,Metropolitan area,Population (2016 est.)[8],MLB,W-L%
0,New York City,20153634,YankeesMets,0.617
1,New York City,20153634,YankeesMets,0.475
2,Los Angeles,13310447,DodgersAngels,0.494
3,Los Angeles,13310447,DodgersAngels,0.564
4,San Francisco Bay Area,6657982,GiantsAthletics,0.599
5,San Francisco Bay Area,6657982,GiantsAthletics,0.451
6,Chicago,9512999,CubsWhite Sox,0.383
7,Chicago,9512999,CubsWhite Sox,0.583
8,Dallas–Fort Worth,7233323,Rangers,0.414
9,"Washington, D.C.",6131977,Nationals,0.506


In [30]:
merge_df=merge_df.dropna()
merge_df=merge_df.reset_index(drop = True)
merge_df.head()

,Metropolitan area,Population (2016 est.)[8],MLB,W-L%
0,New York City,20153634,YankeesMets,0.617
1,New York City,20153634,YankeesMets,0.475
2,Los Angeles,13310447,DodgersAngels,0.494
3,Los Angeles,13310447,DodgersAngels,0.564
4,San Francisco Bay Area,6657982,GiantsAthletics,0.599


In [31]:
merge_df=merge_df.groupby(['Metropolitan area','Population (2016 est.)[8]']).agg('mean')
merge_df.head()

,,W-L%
Metropolitan area,Population (2016 est.)[8],
Atlanta,5789700,0.556
Baltimore,2798886,0.290
Boston,4794447,0.667
Chicago,9512999,0.483
Cincinnati,2165139,0.414


# Question 4
#### For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NFL using 2018 data.

In [32]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


In [33]:
cities['NFL']=cities['NFL'].str.replace('\[.*\]','')
cities.head(55)

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets,YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers,DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders,GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears,CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals[note 10],Wizards[note 11],Capitals
6,Philadelphia,6070500,Eagles,Phillies[note 12],76ers,Flyers[note 13]
7,Boston,4794447,Patriots,Red Sox[note 15],Celtics,Bruins
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild[note 16]
9,Denver,2853077,Broncos,Rockies,Nuggets[note 17],Avalanche[note 18]


In [34]:
nfl_df=nfl_df.drop([0,5,10,15,20,25,30,35])
nfl_df.head()

,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team,year
1,2.1,5,NFL,6.9,3.1,325,111,436,5.2,-1.8,0,11,.688,New England Patriots*,2018
2,-5.2,9,NFL,-7.1,-3.6,433,-114,319,-8.8,-1.7,0,7,.438,Miami Dolphins,2018
3,-0.6,10,NFL,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6,.375,Buffalo Bills,2018
4,-5.9,12,NFL,-6.8,-2.0,441,-108,333,-7.8,-1.1,0,4,.250,New York Jets,2018
6,6.4,6,NFL,6.4,0.6,287,102,389,7.0,0.6,0,10,.625,Baltimore Ravens*,2018


In [35]:
nfl_df[['Metropolitan area','nfl_team']]=nfl_df['team'].str.rsplit(n=1,expand=True)
nfl_df=nfl_df[nfl_df['year']==2018]
nfl_df.head(50)

,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team,year,Metropolitan area,nfl_team
1,2.1,5,NFL,6.9,3.1,325,111,436,5.2,-1.8,0,11,.688,New England Patriots*,2018,New England,Patriots*
2,-5.2,9,NFL,-7.1,-3.6,433,-114,319,-8.8,-1.7,0,7,.438,Miami Dolphins,2018,Miami,Dolphins
3,-0.6,10,NFL,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6,.375,Buffalo Bills,2018,Buffalo,Bills
4,-5.9,12,NFL,-6.8,-2.0,441,-108,333,-7.8,-1.1,0,4,.250,New York Jets,2018,New York,Jets
6,6.4,6,NFL,6.4,0.6,287,102,389,7.0,0.6,0,10,.625,Baltimore Ravens*,2018,Baltimore,Ravens*
7,1.7,6,NFL,4.3,3.9,360,68,428,5.6,1.3,1,9,.594,Pittsburgh Steelers,2018,Pittsburgh,Steelers
8,0.6,8,NFL,-2.1,-1.0,392,-33,359,-0.3,1.7,1,7,.469,Cleveland Browns,2018,Cleveland,Browns
9,-3.4,10,NFL,-5.4,0.0,455,-87,368,-3.4,2.0,0,6,.375,Cincinnati Bengals,2018,Cincinnati,Bengals
11,1.4,5,NFL,5.4,2.4,316,86,402,3.8,-1.5,0,11,.688,Houston Texans*,2018,Houston,Texans*
12,-0.6,6,NFL,5.6,3.9,344,89,433,3.4,-2.2,0,10,.625,Indianapolis Colts+,2018,Indianapolis,Colts+


In [36]:
nfl_df['W']=nfl_df['W'].astype(float)
nfl_df['L']=nfl_df['L'].astype(float)
nfl_df['W/L']=nfl_df['W']/(nfl_df['W']+nfl_df['L'])
nfl_df['Metropolitan area']=nfl_df['Metropolitan area'].replace({'New England':'Boston','Miami':'Miami–Fort Lauderdale','Tennessee':'Nashville','Tampa Bay':'Tampa Bay Area','New York':'New York City','Dallas':'Dallas–Fort Worth','Washington':'Washington, D.C.','San Francisco':'San Francisco Bay Area','Minnesota':'Minneapolis–Saint Paul','Arizona':'Phoenix','Carolina':'Charlotte','Oakland':'San Francisco Bay Area'},regex=True)
merge_df=pd.merge(cities,nfl_df,'outer',on='Metropolitan area')
merge_df=merge_df.drop(['MLB','NBA','NHL','year','League','nfl_team','W','L','SoS','OSRS','PA','PD','PF','MoV','SRS','T','W-L%','DSRS'],axis=1)
merge_df=merge_df.dropna()
merge_df=merge_df.reset_index(drop = True)
merge_df=merge_df.groupby(['Metropolitan area','Population (2016 est.)[8]']).agg('mean')
merge_df.head(50)

,,W/L
Metropolitan area,Population (2016 est.)[8],
Atlanta,5789700,0.437500
Baltimore,2798886,0.625000
Boston,4794447,0.687500
Buffalo,1132804,0.375000
Charlotte,2474314,0.437500
Chicago,9512999,0.750000
Cincinnati,2165139,0.375000
Cleveland,2055612,0.466667
Dallas–Fort Worth,7233323,0.625000
